# Epic remaining cost estimation based on team velocity

Cost estimation for all open (not Completed, Rejected) epics in the project. The model estimates only not completed stories that are assigned to epics in Jira. 
In this approach bugs are not estimated in sprints and they're affecting calculations only trough team velocity. More bugs - velocity is lower and the cost of the epic will be higher and vice versa.

In [26]:
import pandas as pd
from jira import JIRA

jira = JIRA('https://kainos-evolve.atlassian.net')

#load all open epics 
jql = 'project=VXT and type=epic and status not in (Completed, Rejected)'

epicsRaw = jira.search_issues(jql)

epics = pd.DataFrame()
epics['version'] = ''
epics['key'] = ''
epics['type'] = ''
epics['status'] = ''
epics['summary'] = ''

#add epics to dataframe
for issue in epicsRaw:
    #issue may have many versions - in this approach, one version per issue is recommended
    for fixVersion in issue.fields.fixVersions:
        epics = epics.append(
            {'version': fixVersion.name, 
             'key': issue.key,
             'type': issue.fields.issuetype.name,
             'status': issue.fields.status.name,
             'summary': issue.fields.summary,
            }, ignore_index=True)
            
epics

,version,key,type,status,summary
0,Frimley MVP,VXT-3952,Epic,Tech. Scoping,Patient Manager Performance Improvements
1,1.14,VXT-3952,Epic,Tech. Scoping,Patient Manager Performance Improvements
2,GCCG PHASE 1,VXT-3773,Epic,Idea,Additional MVP requirements post release 1.10
3,GCCG PHASE 1,VXT-3690,Epic,Idea,Distributed Patient Record - Phase 1 Improvements
4,ITH POST GSM,VXT-3577,Epic,Awaiting Prioritisation,Form Renderer Refactoring - Improvements
5,GCCG PHASE 1,VXT-3564,Epic,Awaiting Prioritisation,Audit Reports - GCCG Phase 1 [NEW REPORTING]
6,GCCG PHASE 1,VXT-3546,Epic,Idea,Integration engine improvements for GCCG Phase 1
7,GCCG PHASE 1,VXT-3530,Epic,Idea,"Build connectors required for Post-MVP, Phase 1"
8,GCCG PHASE 1,VXT-3527,Epic,Idea,User account enhancements - GCCG Phase 1
9,GCCG PHASE 1,VXT-3524,Epic,Idea,"UX improvements - Post MVP, Phase 1"


Find all issues related to those epics

In [27]:
epicKeys = epics['key'].tolist()
jql = '"Epic Link" in (' + ", ".join(epicKeys) + ')'
jql

'"Epic Link" in (VXT-3952, VXT-3952, VXT-3773, VXT-3690, VXT-3577, VXT-3564, VXT-3546, VXT-3530, VXT-3527, VXT-3524, VXT-3507, VXT-3506, VXT-3497, VXT-3485, VXT-3484, VXT-3376, VXT-3351, VXT-3337, VXT-3292, VXT-3264, VXT-3261, VXT-3260)'

In [28]:
issuesRaw = jira.search_issues(jql)

issues = pd.DataFrame()

issues['epic'] = ''
issues['key'] = ''
issues['type'] = ''
issues['status'] = ''
issues['SP'] = 0
issues['summary'] = ''

for issue in issuesRaw:
    issues = issues.append(
        {
         'key': issue.key,
         'type': issue.fields.issuetype.name,
         'status': issue.fields.status.name,
         'SP': issue.fields.customfield_10005,
         'summary': issue.fields.summary,
         'team' : str(issue.fields.customfield_14200),
         'epic': issue.fields.customfield_10008
        }, ignore_index=True)

#only open issues are calculated
issues = issues.loc[~(issues['status'].isin(['Completed', 'Rejected']))]
issues.sort_values(["epic", 'type', 'status'], inplace=True)

#bugs are not required to be estimated
issues['emptySP'] = ((issues.type == 'Story') & issues.SP.isnull())
issues['notEmptySP'] = (~issues.emptySP)

issues

,epic,key,type,status,SP,summary,team,emptySP,notEmptySP
41,VXT-3337,VXT-3252,Story,Tech Refinement,3.0,Task to get email from FHIR location and enabl...,Custom Controls,False,True
25,VXT-3351,VXT-3494,Story,Awaiting Prioritisation,3.0,Provision dual connectivity to UK pod,Ops Team,False,True
35,VXT-3351,VXT-3356,Story,Awaiting Prioritisation,8.0,Security testing dual connectivity to UK pod,Ops Team,False,True
36,VXT-3351,VXT-3354,Story,Awaiting Prioritisation,NaN,Correct issues found due to dual connectivity ...,Platform Team,True,False
37,VXT-3351,VXT-3352,Story,Awaiting Prioritisation,NaN,Test dual connectivity changes to UK pod,Platform Team,True,False
33,VXT-3376,VXT-3378,Story,Tech Refinement,NaN,Design ePCR template,None,True,False
34,VXT-3376,VXT-3377,Story,Tech Refinement,NaN,Create generic ePCR form wireframes,None,True,False
16,VXT-3484,VXT-3589,Story,Idea,NaN,Move form redirect functionality into form ren...,Web Team,True,False
44,VXT-3506,VXT-3233,Story,Awaiting Prioritisation,30.0,Placeholder for additional custom controls,None,False,True
46,VXT-3506,VXT-3205,Story,Awaiting Prioritisation,1.0,Online awareness for postcode lookup / address...,None,False,True


In [29]:
import numpy as np
#in python we can treat True as 1 and False as 0 so simple sum suffice to calculate count of 
#estimated and not estimated issues in each epic
aggrIssues = issues.groupby(['epic']).agg({'emptySP':'sum','notEmptySP':'sum', 'SP': 'sum'})
aggrIssues['estimatedPerc'] = np.ceil(aggrIssues.notEmptySP / (aggrIssues.notEmptySP + aggrIssues.emptySP) * 100)
aggrIssues = aggrIssues.reset_index()

aggrIssues

,epic,notEmptySP,emptySP,SP,estimatedPerc
0,VXT-3337,1.0,0.0,3.0,100.0
1,VXT-3351,2.0,2.0,11.0,50.0
2,VXT-3376,0.0,2.0,0.0,0.0
3,VXT-3484,0.0,1.0,0.0,0.0
4,VXT-3506,3.0,0.0,36.0,100.0
5,VXT-3524,0.0,2.0,0.0,0.0
6,VXT-3527,0.0,1.0,0.0,0.0
7,VXT-3546,3.0,0.0,11.0,100.0
8,VXT-3564,4.0,2.0,1.0,67.0
9,VXT-3690,3.0,1.0,0.0,75.0


<div class="alert alert-block alert-success">
Set max & min estimated velocity of the team and team sprint costs
</div>

In [30]:
minVelocity = 10
avgVelocity = 15
maxVelocity = 20
#assuming that a team have 4 developers and each one salary is 1000 / week and we have 2W sprints
sprintCosts = 4 * 1000 * 2

In [31]:
del aggrIssues['emptySP']
del aggrIssues['notEmptySP']

#if the epic's stories are not estimated
aggrIssues['minCost'] = 'Data not sufficient to estimate'
aggrIssues['avgCost'] = 'Data not sufficient to estimate'
aggrIssues['maxCost'] = 'Data not sufficient to estimate'

#only calculate costs for fully estimated epics
aggrIssues.loc[((aggrIssues['estimatedPerc'] == 100)), ['minCost']] = np.ceil(aggrIssues.SP / maxVelocity * sprintCosts)
aggrIssues.loc[((aggrIssues['estimatedPerc'] == 100)), ['avgCost']] = np.ceil(aggrIssues.SP / avgVelocity * sprintCosts)
aggrIssues.loc[((aggrIssues['estimatedPerc'] == 100)), ['maxCost']] = np.ceil(aggrIssues.SP / minVelocity * sprintCosts)

#aggrIssues

In [32]:
#load epic descriptions from Jira
epicNames = []
for epicKey in aggrIssues['epic']:
    epic = jira.issue(epicKey)
    epicNames.append(epic.fields.summary)
    
epicNames = pd.Series(epicNames)
aggrIssues['summary'] = epicNames.values



    

In [35]:
#change column order
cols = ['epic', 'summary', 'SP', 'estimatedPerc', 'minCost', 'avgCost', 'maxCost']
aggrIssues = aggrIssues[cols]


aggrIssues

,epic,summary,SP,estimatedPerc,minCost,avgCost,maxCost
0,VXT-3337,Custom task for sending emails to the given lo...,3.0,100.0,1200,1600,2400
1,VXT-3351,Dual connectivity to UK pod - Internet/N3,11.0,50.0,Data not sufficient to estimate,Data not sufficient to estimate,Data not sufficient to estimate
2,VXT-3376,ePCR template productisation,0.0,0.0,Data not sufficient to estimate,Data not sufficient to estimate,Data not sufficient to estimate
3,VXT-3484,iOS epcr MVP bugs,0.0,0.0,Data not sufficient to estimate,Data not sufficient to estimate,Data not sufficient to estimate
4,VXT-3506,Custom Controls for ePCR MVP (ePCR),36.0,100.0,14400,19200,28800
5,VXT-3524,"UX improvements - Post MVP, Phase 1",0.0,0.0,Data not sufficient to estimate,Data not sufficient to estimate,Data not sufficient to estimate
6,VXT-3527,User account enhancements - GCCG Phase 1,0.0,0.0,Data not sufficient to estimate,Data not sufficient to estimate,Data not sufficient to estimate
7,VXT-3546,Integration engine improvements for GCCG Phase 1,11.0,100.0,4400,5867,8800
8,VXT-3564,Audit Reports - GCCG Phase 1 [NEW REPORTING],1.0,67.0,Data not sufficient to estimate,Data not sufficient to estimate,Data not sufficient to estimate
9,VXT-3690,Distributed Patient Record - Phase 1 Improvements,0.0,75.0,Data not sufficient to estimate,Data not sufficient to estimate,Data not sufficient to estimate
